In [ ]:
from VR_Trajectory_analysis import *
df_cut_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/color_bias_geometry3/df_cut.pkl'
results_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/color_bias_geometry3/results_df.pkl'
df_cut = pd.read_pickle(df_cut_path_pickle)
results_df = pd.read_pickle(results_df_path_pickle)

In [ ]:
df_cut.columns

In [ ]:
results_df.columns

In [ ]:
#df_cut already has all the columns from results_df...
'''df_joined = pd.merge(
    df_cut,
    results_df[['UniqueTrialID', 'FirstReachedGoal']],
    on='UniqueTrialID',
    how='left'
)'''
df_joined = df_cut
#add angle column to df_joined
df_joined['Angle'] = df_joined['ConfigFile'].apply(lambda x: parse_angle_from_config(x, default_angle=999.0))

In [ ]:
df_joined.columns

In [ ]:
# -----------------------------------------------------------------
#  Trajectories for all  "BlueCylinder_BlueGreenCylinder" configs
#  shown in a grid, one panel per Angle
# -----------------------------------------------------------------
import math, re
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec

# 1.  The joined dataframe is assumed to exist:  df_joined

# 2.  Keep only the configurations we care about ---------------------------
accept_tags = ("BlueCylinder_BlueGreenCylinder",
               "Blue_BlueGreen")

df_filt = df_joined[df_joined['ConfigFile'].str.contains('|'.join(accept_tags))]

# make sure Angle is numeric so the sort works
df_filt['Angle'] = pd.to_numeric(df_filt['Angle'], errors='coerce')

angles = sorted(df_filt['Angle'].dropna().unique())
if not angles:
    raise ValueError("No matching configs with an Angle column found.")

# 3.  Build grid layout -----------------------------------------------------
n_panels = len(angles)
n_cols   = 3                               # change to taste
n_rows   = math.ceil(n_panels / n_cols)

fig = plt.figure(figsize=(n_cols*4, n_rows*3.75))
gs  = gridspec.GridSpec(n_rows, n_cols, wspace=.25, hspace=.35)

# 4.  One sub-plot per Angle -------------------------------------------------
for idx, angle in enumerate(angles):
    ax = fig.add_subplot(gs[idx])
    g  = df_filt[df_filt['Angle'] == angle]

    # trajectories
    for trial_id, td in g.groupby('UniqueTrialID'):
        ax.plot(td['GameObjectPosX'], td['GameObjectPosZ'], alpha=.1)

    # basic aesthetics
    ax.set_xlim(-40, 40)
    ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')

    ax.set_xticks([-60, -40, -20, 0, 20, 40, 60])
    ax.set_yticks([0, 20, 40, 60])
    ax.tick_params(labelsize=8)

    ax.set_title(f"{int(angle)}°", fontsize=10)

    # left / right counts & ratios
    res   = results_df.loc[results_df['UniqueTrialID'].isin(g['UniqueTrialID'])]
    left  = (res['FirstReachedGoal'] == 'left').sum()
    right = (res['FirstReachedGoal'] == 'right').sum()
    tot   = left + right
    txt   = f"L: {left} ({left/tot:.2f})\nR: {right} ({right/tot:.2f})" if tot else "no goals"

    ax.text(.02, .98, txt, transform=ax.transAxes, va='top',
            fontsize=8, bbox=dict(boxstyle="round", fc="w", ec=".5"))

# hide any leftover empty cells
for j in range(idx+1, n_rows*n_cols):
    fig.add_subplot(gs[j]).axis('off')

fig.suptitle("Trajectories – BlueCylinder_BlueGreenCylinder", y=.92, fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------------
#  Trajectories for all  "BlueCylinder_BlueGreenCylinder" configs
#  shown in a grid, one panel per Angle
# -----------------------------------------------------------------
import math, re
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec
from matplotlib.lines import Line2D  # for legend

# 1.  The joined dataframe is assumed to exist:  df_joined

# 2.  Keep only the configurations we care about ---------------------------
accept_tags = ("BlueCylinder_BlueGreenCylinder", "Blue_BlueGreen")
df_filt = df_joined[df_joined['ConfigFile'].str.contains('|'.join(accept_tags))]

# make sure Angle is numeric so the sort works
df_filt['Angle'] = pd.to_numeric(df_filt['Angle'], errors='coerce')

angles = sorted(df_filt['Angle'].dropna().unique())
if not angles:
    raise ValueError("No matching configs with an Angle column found.")

# 3.  Build grid layout -----------------------------------------------------
n_panels = len(angles)
n_cols   = 3                               # change to taste
n_rows   = math.ceil(n_panels / n_cols)

fig = plt.figure(figsize=(n_cols*12, n_rows*11.25))
gs  = gridspec.GridSpec(n_rows, n_cols, wspace=.25, hspace=.35)

# 4.  One sub-plot per Angle -------------------------------------------------
for idx, angle in enumerate(angles):
    ax = fig.add_subplot(gs[idx])
    g  = df_filt[df_filt['Angle'] == angle]

    # Get reach direction per trial
    res = results_df.loc[results_df['UniqueTrialID'].isin(g['UniqueTrialID'])]
    reach_map = dict(zip(res['UniqueTrialID'], res['FirstReachedGoal']))

    # trajectories with color coding
    for trial_id, td in g.groupby('UniqueTrialID'):
        goal = reach_map.get(trial_id, None)
        if goal == 'left':
            color = 'blue'
        elif goal == 'right':
            color = 'green'
        else:
            color = 'gray'  # fallback for missing or other labels

        ax.plot(td['GameObjectPosX'], td['GameObjectPosZ'], alpha=.2, color=color)

    # basic aesthetics
    ax.set_xlim(-40, 40)
    ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')

    ax.set_xticks([-60, -40, -20, 0, 20, 40, 60])
    ax.set_yticks([0, 20, 40, 60])
    ax.tick_params(labelsize=8)

    ax.set_title(f"{int(angle)}°", fontsize=10)

    # left / right counts & ratios
    left  = (res['FirstReachedGoal'] == 'left').sum()
    right = (res['FirstReachedGoal'] == 'right').sum()
    tot   = left + right
    txt   = f"L: {left} ({left/tot:.2f})\nR: {right} ({right/tot:.2f})" if tot else "no goals"

    ax.text(.02, .98, txt, transform=ax.transAxes, va='top',
            fontsize=8, bbox=dict(boxstyle="round", fc="w", ec=".5"))

# hide any leftover empty cells
for j in range(idx+1, n_rows*n_cols):
    fig.add_subplot(gs[j]).axis('off')

# Add a shared legend
legend_elements = [
    Line2D([0], [0], color='blue',  lw=2, label='Reached Left'),
    Line2D([0], [0], color='green', lw=2, label='Reached Right'),
    Line2D([0], [0], color='gray',  lw=2, label='Unknown')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3)

fig.suptitle("Trajectories – BlueCylinder_BlueGreenCylinder", y=.92, fontsize=16)
plt.tight_layout(rect=[0, 0.05, 1, 0.94])  # leave room for legend and title
plt.show()


In [ ]:
# ------------------------------------------------------------
# Preferred-goal reach vs platform angle   (all two-goal configs)
# ------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint   # pip install statsmodels

# ── 1.  Merge goal info into movement dataframe ──────────────────────────
# (uncomment if you haven’t created df_joined yet)
# df_joined = pd.merge(
#     df_cut,
#     results_df[['UniqueTrialID', 'FirstReachedGoal']],
#     on='UniqueTrialID',
#     how='left'
# )

# ensure Angle is numeric
df_joined['Angle'] = pd.to_numeric(df_joined['Angle'], errors='coerce')

# ── 2.  Per-config preferred-side counts ─────────────────────────────────
records = []

for cfg, g in df_joined.groupby('ConfigFile'):

    # skip single-goal layouts if you keep such a list
    if 'center_only_configs' in globals() and cfg in center_only_configs:
        continue

    # ---- goal counts for THIS config ------------------------------------
    res   = results_df.loc[results_df['UniqueTrialID'].isin(g['UniqueTrialID'])]
    left  = (res['FirstReachedGoal'] == 'left').sum()
    right = (res['FirstReachedGoal'] == 'right').sum()
    n     = left + right
    if n == 0:
        continue

    # ---- which platform is on the left / right? -------------------------
    if   "BlueGreenCylinder_BlueCylinder" in cfg or "BlueGreen_Blue" in cfg:
        label, colour, pref_cnt = 'BG→B', 'tab:blue',  left   # preferred = Left
    elif "BlueCylinder_BlueGreenCylinder" in cfg or "Blue_BlueGreen" in cfg:
        label, colour, pref_cnt = 'B→BG', 'tab:green', right  # preferred = Right
    else:
        label, colour, pref_cnt = 'other', 'tab:orange', left # symmetrical

    # ---- angle ----------------------------------------------------------
    angle_vals = g['Angle'].dropna().unique()
    if len(angle_vals) != 1:
        print(f'⚠️  Angle ambiguous → skipped: {cfg}')
        continue
    angle = float(angle_vals[0])

    records.append(dict(
        angle=angle, label=label, colour=colour,
        pref_cnt=pref_cnt, n_trials=n
    ))

# ── 3.  Aggregate over *all* configs that share the same angle ───────────
df  = pd.DataFrame(records)
agg = (df.groupby(['angle', 'label', 'colour'], as_index=False)
         .agg(pref_cnt=('pref_cnt', 'sum'),
              n_trials =('n_trials', 'sum')))

# Wilson CIs & preferred-goal proportion
lo, hi = [], []
for s, n in zip(agg['pref_cnt'], agg['n_trials']):
    l, h = proportion_confint(s, n, alpha=.05, method='wilson')
    lo.append(l); hi.append(h)

agg['p_pref']  = agg['pref_cnt'] / agg['n_trials']
agg['lo_pref'] = lo
agg['hi_pref'] = hi
# ------------------------------------------------------------------------

# ── 4.  Plot ─────────────────────────────────────────────────────────────
label_map = {'BG→B': 'Left', 'B→BG': 'Right', 'other': 'Symmetric'}

plt.figure(figsize=(6, 4))

for lbl, grp in agg.groupby('label'):
    plt.scatter(grp['angle'], grp['p_pref'],
                c=grp['colour'],
                label=label_map[lbl],
                s=60, alpha=.9)

    grp = grp.sort_values('angle')
    plt.plot(grp['angle'], grp['p_pref'],
             c=grp['colour'].iloc[0], lw=1, alpha=.4)

    plt.errorbar(grp['angle'], grp['p_pref'],
                 yerr=[grp['p_pref'] - grp['lo_pref'],
                       grp['hi_pref'] - grp['p_pref']],
                 fmt='none',
                 ecolor=grp['colour'].iloc[0],
                 alpha=.5, capsize=3)

plt.axhline(.5, ls='--', alpha=.4)
plt.xlabel('Platform angle (deg)')
plt.ylabel('Fraction of runs reaching preferred goal')
plt.title('Preferred-goal reach vs. platform angle   (95 % Wilson CIs)')
plt.legend(title='Preferred Goal')
plt.tight_layout()
plt.show()

In [ ]:
# ------------------------------------------------------------
# Preferred-goal reach vs platform angle   (all two-goal configs)
# ------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint   # pip install statsmodels

# ── 1.  Merge goal info into movement dataframe ──────────────────────────
# (uncomment if you haven’t created df_joined yet)
# df_joined = pd.merge(
#     df_cut,
#     results_df[['UniqueTrialID', 'FirstReachedGoal']],
#     on='UniqueTrialID',
#     how='left'
# )

# ensure Angle is numeric
df_joined['Angle'] = pd.to_numeric(df_joined['Angle'], errors='coerce')

# ── 2.  Per-config preferred-side counts ─────────────────────────────────
records = []

for cfg, g in df_joined.groupby('ConfigFile'):

    # skip single-goal layouts if you keep such a list
    if 'center_only_configs' in globals() and cfg in center_only_configs:
        continue

    # ---- goal counts for THIS config ------------------------------------
    res   = results_df.loc[results_df['UniqueTrialID'].isin(g['UniqueTrialID'])]
    left  = (res['FirstReachedGoal'] == 'left').sum()
    right = (res['FirstReachedGoal'] == 'right').sum()
    n     = left + right
    if n == 0:
        continue

    # ---- which platform is on the left / right? -------------------------
    if "BlueGreenCylinder_BlueCylinder" in cfg or "BlueGreen_Blue" in cfg:
        label, colour, pref_cnt = 'asymmetric', 'tab:blue', left
    elif "BlueCylinder_BlueGreenCylinder" in cfg or "Blue_BlueGreen" in cfg:
        label, colour, pref_cnt = 'asymmetric', 'tab:blue', right
    # ---- symmetrical configs --------------------------------------------
    else:
        label, colour, pref_cnt = 'other', 'tab:orange', left # symmetrical

    # ---- angle ----------------------------------------------------------
    angle_vals = g['Angle'].dropna().unique()
    if len(angle_vals) != 1:
        print(f'⚠️  Angle ambiguous → skipped: {cfg}')
        continue
    angle = float(angle_vals[0])

    records.append(dict(
        angle=angle, label=label, colour=colour,
        pref_cnt=pref_cnt, n_trials=n
    ))

# ── 3.  Aggregate over *all* configs that share the same angle ───────────
df  = pd.DataFrame(records)
agg = (df.groupby(['angle', 'label', 'colour'], as_index=False)
         .agg(pref_cnt=('pref_cnt', 'sum'),
              n_trials =('n_trials', 'sum')))

# Wilson CIs & preferred-goal proportion
lo, hi = [], []
for s, n in zip(agg['pref_cnt'], agg['n_trials']):
    l, h = proportion_confint(s, n, alpha=.05, method='wilson')
    lo.append(l); hi.append(h)

agg['p_pref']  = agg['pref_cnt'] / agg['n_trials']
agg['lo_pref'] = lo
agg['hi_pref'] = hi
# ------------------------------------------------------------------------

# ── 4.  Plot ─────────────────────────────────────────────────────────────
label_map = {'asymmetric': 'Asymmetric', 'other': 'Symmetric'}


plt.figure(figsize=(6, 4))

for lbl, grp in agg.groupby('label'):
    plt.scatter(grp['angle'], grp['p_pref'],
                c=grp['colour'],
                label=label_map[lbl],
                s=60, alpha=.9)

    grp = grp.sort_values('angle')
    plt.plot(grp['angle'], grp['p_pref'],
             c=grp['colour'].iloc[0], lw=1, alpha=.4)

    plt.errorbar(grp['angle'], grp['p_pref'],
                 yerr=[grp['p_pref'] - grp['lo_pref'],
                       grp['hi_pref'] - grp['p_pref']],
                 fmt='none',
                 ecolor=grp['colour'].iloc[0],
                 alpha=.5, capsize=3)

plt.axhline(.5, ls='--', alpha=.4)
plt.xlabel('Platform angle (deg)')
plt.ylabel('Fraction of runs reaching preferred goal')
plt.title('Preferred-goal reach vs. platform angle   (95 % Wilson CIs)')
plt.legend(title='Preferred Goal')
plt.tight_layout()
plt.show()

In [ ]:
df_joined['FlyID'].nunique()  # to check unique fly IDs in the joined dataframe

There is one Ant without colony ID

In [ ]:
df_cut["ColonyID"] = df_cut["FlyID"].str[0]

In [ ]:
import re, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ── 0) START: merge once to get the goal info if not already there ──────────────
if 'FirstReachedGoal' not in df_cut.columns:
    df_joined = (
        pd.merge(
            df_cut,                                  # has FlyID etc.
            results_df[['UniqueTrialID', 'FirstReachedGoal']],
            on='UniqueTrialID',
            how='left')
          .copy()
    )
else:
    df_joined = df_cut.copy()

# If the merge produced a suffixed name, normalise it
if 'FirstReachedGoal' not in df_joined.columns:
    cand = [c for c in df_joined.columns if c.startswith('FirstReachedGoal')]
    if not cand:
        raise KeyError("No 'FirstReachedGoal' column found after merge.")
    df_joined.rename(columns={cand[0]: 'FirstReachedGoal'}, inplace=True)

# ── 1) keep *two-goal* configurations only ─────────────────────────────────────
two_goal_mask = (
    df_joined['ConfigFile'].str.contains("BlueGreenCylinder_BlueCylinder") |
    df_joined['ConfigFile'].str.contains("BlueCylinder_BlueGreenCylinder")
)
df_pref = df_joined.loc[two_goal_mask].copy()

# ── 2) vectorised “preferred side” lookup ──────────────────────────────────────
pref_side = np.where(
    df_pref['ConfigFile'].str.contains("BlueGreenCylinder_BlueCylinder"),
    'left',          # BG→B
    'right'          # B→BG
)
df_pref['is_pref'] = (df_pref['FirstReachedGoal'] == pref_side)

# ── 3) per-fly fraction (drop flies with < 3 trials per config) ────────────────
fly_cfg = (
    df_pref
      .groupby(['ConfigFile', 'FlyID'])
      .agg(ratio_pref=('is_pref', 'mean'), n_trials=('is_pref', 'size'))
      .reset_index()
      .query('n_trials >= 3')
)

# ── 4) extract the numeric angle for the x-axis ────────────────────────────────
angle_pat = re.compile(r'(\d+)[°d]')
fly_cfg['angle'] = (
    fly_cfg['ConfigFile'].str.extract(angle_pat)[0].astype(int)
)

# ── 5) violin plot with Matplotlib (no seaborn needed) ─────────────────────────
plt.figure(figsize=(10, 5), dpi=120)

# build a list of datasets ordered by angle
angles = sorted(fly_cfg['angle'].unique())
data   = [fly_cfg.loc[fly_cfg['angle'] == a, 'ratio_pref'] for a in angles]

parts = plt.violinplot(data, positions=angles, widths=3,
                       showmeans=False, showmedians=False, showextrema=False)

# cosmetic tweaks
for pc in parts['bodies']:
    pc.set_facecolor('#5A9EBB')
    pc.set_edgecolor('k')
    pc.set_alpha(0.6)

# median + quartile markers
for xpos, vals in zip(angles, data):
    if len(vals) == 0:          # safety
        continue
    q1, med, q3 = np.percentile(vals, [25, 50, 75])
    plt.scatter([xpos]*3, [q1, med, q3], c='k', s=12, zorder=3)
    plt.plot([xpos, xpos], [q1, q3], c='k', lw=1)   # IQR bar

plt.axhline(0.5, ls='--', c='k', lw=.8, alpha=.5)
plt.xlabel('Platform angle (°)')
plt.ylabel('Fraction of that fly’s trials\nending in preferred goal')
plt.title('Per-fly preference ratios by platform angle')
plt.xlim(min(angles)-5, max(angles)+5)
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calculate number of unique trials per FlyID
counts = df_cut.groupby('FlyID')['UniqueTrialID'].nunique()

# Plot histogram
plt.figure(figsize=(8, 4))
plt.hist(counts, bins='auto')  # default settings adhere to style rules
plt.xlabel('Number of unique trials per Fly')
plt.ylabel('Number of flies')
plt.title('Histogram of trial counts per FlyID')
plt.tight_layout()
plt.show()

In [ ]:
# Extract first GoalReachedTime per trial
decision_times = (
    df_joined.groupby('UniqueTrialID')
             .agg(GoalReachedTime=('GoalReachedTime', 'first'),
                  FlyID=('FlyID', 'first'))  # Include FlyID for later analysis
             .reset_index()
)


In [ ]:
import seaborn as sns

plt.figure(figsize=(6, 4))
sns.histplot(decision_times['GoalReachedTime'], bins=30, kde=True)
plt.ylabel('Trial Count')
plt.xlabel('Decision Time (s)')
plt.title('Distribution of Decision Times')
plt.tight_layout()
plt.show()


In [ ]:
# 1. Compute mean decision time per FlyID
fly_means = (
    decision_times.groupby('FlyID')['GoalReachedTime']
                  .mean()
                  .sort_values()
)

# 2. Convert FlyID to ordered category
decision_times['FlyID'] = pd.Categorical(
    decision_times['FlyID'],
    categories=fly_means.index,
    ordered=True
)

# 3. Plot sorted boxplot
plt.figure(figsize=(20, 8))
sns.boxplot(data=decision_times, x='FlyID', y='GoalReachedTime')
plt.xticks(rotation=80)
plt.ylabel('Decision Time (s)')
plt.title('Decision Time Distribution per Ant (Sorted by Mean)')
plt.tight_layout()
plt.show()


In [ ]:
# 1. Count number of trials per FlyID
trial_counts = (
    decision_times['FlyID']
    .value_counts()
    .reindex(fly_means.index)  # same order as boxplot
)

# 2. Plot
plt.figure(figsize=(20, 8))
sns.barplot(x=trial_counts.index, y=trial_counts.values)
plt.xticks(rotation=80)
plt.ylabel('Number of Trials')
plt.title('Number of Trials per Ant (Sorted by Mean Decision Time)')
plt.tight_layout()
plt.show()


In [ ]:
def infer_correct_goal(cfg):
    if "BlueGreenCylinder_BlueCylinder" in cfg or "BlueGreen_Blue" in cfg:
        return 'left'
    elif "BlueCylinder_BlueGreenCylinder" in cfg or "Blue_BlueGreen" in cfg:
        return 'right'
    else:
        return np.nan  # ambiguous/symmetric configs

df_joined['CorrectGoal'] = df_joined['ConfigFile'].apply(infer_correct_goal)


In [ ]:
import numpy as np

df_joined['is_correct'] = np.where(
    df_joined['CorrectGoal'].isna(),
    np.nan,  # Mark as undefined, not True or False
    df_joined['FirstReachedGoal'] == df_joined['CorrectGoal']
)


In [ ]:
# Only include trials with a defined correct answer
df_with_defined_goal = df_joined.dropna(subset=['is_correct'])

# Accuracy per ant (only valid trials)
accuracy_by_ant = (
    df_with_defined_goal.groupby('FlyID')['is_correct']
                        .mean()
                        .reset_index(name='accuracy')
)



In [ ]:
# Filter to valid trials with a defined correct goal
df_valid = df_joined.dropna(subset=['is_correct'])

# Compute accuracy per FlyID
accuracy_by_ant = (
    df_valid.groupby('FlyID')['is_correct']
            .mean()
            .reset_index(name='accuracy')
            .sort_values('accuracy')  # sort by accuracy if desired
)

# Plot
plt.figure(figsize=(20, 8))
sns.barplot(data=accuracy_by_ant, x='FlyID', y='accuracy')
plt.ylim(0, 1)
plt.xticks(rotation=80)
plt.ylabel('Accuracy')
plt.title('Accuracy per Ant (only trials with defined correct goal)')
plt.tight_layout()
plt.show()


In [ ]:

# 1. Group all trials (no filtering)
df_valid_all = df_joined.dropna(subset=['is_correct'])

# 2. Accuracy per Ant × Angle
acc_all = (
    df_valid_all.groupby(['FlyID', 'Angle'])['is_correct']
                .mean()
                .reset_index(name='accuracy')
)

# 3. Total trials per ant (for sorting)
trial_counts = (
    df_valid_all.groupby('FlyID')['UniqueTrialID']
                .nunique()
                .reset_index(name='total_trials')
)

# 4. Merge trial count into accuracy dataframe
acc_all = pd.merge(acc_all, trial_counts, on='FlyID')

# 5. Sort ants by trial count
sorted_ants = trial_counts.sort_values('total_trials', ascending=False)['FlyID']

# 6. Set FlyID as ordered category
acc_all['FlyID'] = pd.Categorical(acc_all['FlyID'], categories=sorted_ants, ordered=True)

# 7. Pivot to wide format for heatmap
pivot = acc_all.pivot(index='FlyID', columns='Angle', values='accuracy')

# 8. Plot heatmap (no text annotations)
plt.figure(figsize=(12, 18))
sns.heatmap(pivot, annot=False, cmap="viridis", vmin=0, vmax=1,
            cbar_kws={'label': 'Accuracy'})
plt.title('Accuracy per Ant per Angle (Sorted by Total Trials)')
plt.ylabel('Ant (FlyID)')
plt.xlabel('Platform Angle (deg)')
plt.tight_layout()
plt.show()



In [ ]:
# Reuse total trial counts
top_20_ants = (
    df_valid_all.groupby('FlyID')['UniqueTrialID']
                .nunique()
                .sort_values(ascending=False)
                .head(20)
                .index
)
acc_top20 = acc_all[acc_all['FlyID'].isin(top_20_ants)]
acc_top20['FlyID'] = pd.Categorical(
    acc_top20['FlyID'],
    categories=top_20_ants,
    ordered=True
)
plt.figure(figsize=(12, 6))
sns.lineplot(data=acc_top20, x='Angle', y='accuracy', hue='FlyID', marker='o')
plt.ylim(0, 1)
plt.ylabel('Accuracy')
plt.xlabel('Platform Angle (deg)')
plt.title('Accuracy across Angles (Top 20 Ants by Trial Count)')
plt.legend(title='Ant (FlyID)', bbox_to_anchor=(1.05, 1), loc='upper left', ncol=2)
plt.tight_layout()
plt.show()


In [ ]:
# Accuracy at 180° per ant
acc_at_180 = (
    acc_top20[acc_top20['Angle'] == 180]
    .set_index('FlyID')['accuracy']
)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import numpy as np

# Normalize for color mapping
norm = plt.Normalize(vmin=0, vmax=1)  # accuracy range 0–1
cmap = cm.get_cmap('viridis')  # or 'coolwarm', 'plasma', etc.


In [ ]:
plt.figure(figsize=(12, 6))

# Plot one ant at a time
for ant, group in acc_top20.groupby('FlyID'):
    color_val = acc_at_180.get(ant, np.nan)

    # Skip if no 180° data
    if np.isnan(color_val):
        continue

    color = cmap(norm(color_val))

    # Sort by angle to ensure clean line
    group_sorted = group.sort_values('Angle')

    plt.plot(group_sorted['Angle'], group_sorted['accuracy'],
             marker='o', label=ant, color=color, linewidth=2, alpha=0.9)

# Colorbar to show accuracy scale
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # dummy array for colorbar
cbar = plt.colorbar(sm)
cbar.set_label('Accuracy at 180°')

plt.ylim(0, 1)
plt.ylabel('Accuracy')
plt.xlabel('Platform Angle (deg)')
plt.title('Accuracy Across Angles\n(Top 20 Ants Colored by Accuracy at 180°)')
plt.tight_layout()
plt.show()


In [ ]:
# Use previously computed accuracy at 180°
acc_at_180 = (
    acc_top20[acc_top20['Angle'] == 180]
    .set_index('FlyID')['accuracy']
)

# Option A: Manual thresholds (adjust as needed)
def accuracy_group(val):
    if val >= 0.71:
        return 'High'
    elif val >= 0.4:
        return 'Low'
    else:
        return 'Low'

acc_group = acc_at_180.apply(accuracy_group)

# Add group info to acc_top20
acc_top20['accuracy_group'] = acc_top20['FlyID'].map(acc_group)


In [ ]:
plt.figure(figsize=(12, 6))

# Define group colors
group_colors = {'High': 'green', 'Medium': 'orange', 'Low': 'red'}

# Plot each ant, colored by its group
for ant, group in acc_top20.groupby('FlyID'):
    g = group.sort_values('Angle')
    grp = group['accuracy_group'].iloc[0]  # all rows from one ant, so it's safe
    color = group_colors.get(grp, 'gray')

    plt.plot(g['Angle'], g['accuracy'],
             color=color, linewidth=1.5, alpha=0.2)

# Overlay mean lines per group
for grp, df_grp in acc_top20.groupby('accuracy_group'):
    mean_curve = (
        df_grp.groupby('Angle')['accuracy']
              .mean()
              .reset_index()
              .sort_values('Angle')
    )
    plt.plot(mean_curve['Angle'], mean_curve['accuracy'],
             label=f'{grp} (n={df_grp["FlyID"].nunique()})',
             color=group_colors[grp], linewidth=3)

plt.ylim(0.5, 1)
plt.ylabel('Accuracy')
plt.xlabel('Platform Angle (deg)')
plt.title('Accuracy Across Angles (Top 20 Ants)\nLines Colored by 180° Accuracy Group')
plt.legend(title='180° Accuracy Group')
plt.tight_layout()
plt.show()


In [ ]:
# Pick 3 random trial IDs with complete data
sample_ids = df_joined['UniqueTrialID'].dropna().unique()[0:3]

# Get those trajectories
df_sample = df_joined[df_joined['UniqueTrialID'].isin(sample_ids)]


In [ ]:
df_sample

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
for i, trial_id in enumerate(sample_ids):
    plt.subplot(1, 3, i + 1)
    trj = df_sample[df_sample['UniqueTrialID'] == trial_id]
    plt.plot(trj['GameObjectPosX'], trj['GameObjectPosZ'], lw=1.5)
 
    plt.title(f"Trial: {trial_id}")
    plt.axis('equal')
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
def compute_path_efficiency(trj):
    x = trj['GameObjectPosX'].values
    y = trj['GameObjectPosZ'].values

    # Total path length
    deltas = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    path_length = deltas.sum()

    # Euclidean distance from start to end
    euclidean_dist = np.sqrt((x[-1] - x[0])**2 + (y[-1] - y[0])**2)

    if path_length == 0:
        return np.nan
    return euclidean_dist / path_length  # ∈ [0, 1]


In [ ]:
for trial_id in sample_ids:
    trj = df_sample[df_sample['UniqueTrialID'] == trial_id]
    eff = compute_path_efficiency(trj)
    print(f"Trial {trial_id} → path efficiency: {eff:.3f}")


In [ ]:
from scipy.signal import savgol_filter
def smooth_trajectory(trj, window_length=11, polyorder=2):
    # Ensure valid window size
    if len(trj) < window_length:
        return trj[['GameObjectPosX', 'GameObjectPosZ']].values

    x = trj['GameObjectPosX'].values
    z = trj['GameObjectPosZ'].values

    x_smooth = savgol_filter(x, window_length, polyorder)
    z_smooth = savgol_filter(z, window_length, polyorder)

    return np.column_stack((x_smooth, z_smooth))


In [ ]:
trial_id = sample_ids[2]
trj = df_sample[df_sample['UniqueTrialID'] == trial_id]

smoothed = smooth_trajectory(trj, window_length=240, polyorder=2)

plt.figure(figsize=(6, 6))
plt.plot(trj['GameObjectPosX'], trj['GameObjectPosZ'], label='Raw', alpha=0.5, lw=1.5)
plt.plot(smoothed[:, 0], smoothed[:, 1], label='Smoothed', lw=2)
plt.scatter(trj['GameObjectPosX'].iloc[0], trj['GameObjectPosZ'].iloc[0], c='green', label='Start')
plt.scatter(trj['GameObjectPosX'].iloc[-1], trj['GameObjectPosZ'].iloc[-1], c='red', label='End')
plt.title(f'Trajectory Smoothing – Trial {trial_id}')
plt.axis('equal')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import random

# Get 20 random trial IDs
unique_ids = df_joined['UniqueTrialID'].dropna().unique()
sample_ids = random.sample(list(unique_ids), 20)
from scipy.signal import savgol_filter

def smooth_trajectory(trj, window_length=240, polyorder=2):
    if len(trj) < window_length:
        return trj[['GameObjectPosX', 'GameObjectPosZ']].values
    x = trj['GameObjectPosX'].values
    z = trj['GameObjectPosZ'].values
    x_smooth = savgol_filter(x, window_length, polyorder)
    z_smooth = savgol_filter(z, window_length, polyorder)
    return np.column_stack((x_smooth, z_smooth))
plt.figure(figsize=(16, 12))
for i, trial_id in enumerate(sample_ids):
    trj = df_joined[df_joined['UniqueTrialID'] == trial_id]
    smoothed = smooth_trajectory(trj, window_length=480, polyorder=2)

    plt.subplot(4, 5, i + 1)
    plt.plot(trj['GameObjectPosX'], trj['GameObjectPosZ'], alpha=0.5, lw=1.2, label='Raw')
    plt.plot(smoothed[:, 0], smoothed[:, 1], lw=2, label='Smoothed')
    plt.scatter(trj['GameObjectPosX'].iloc[0], trj['GameObjectPosZ'].iloc[0], c='green', s=10)
    plt.scatter(trj['GameObjectPosX'].iloc[-1], trj['GameObjectPosZ'].iloc[-1], c='red', s=10)
    plt.axis('equal')
    plt.title(f'{trial_id}', fontsize=8)
    plt.xticks([])
    plt.yticks([])

plt.suptitle('Raw vs Smoothed Trajectories (Savitzky-Golay)', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()


In [ ]:
import random
from scipy.signal import savgol_filter
import numpy as np
import matplotlib.pyplot as plt

# Define path efficiency function
def path_efficiency(x, z):
    if len(x) < 2:
        return np.nan
    deltas = np.sqrt(np.diff(x)**2 + np.diff(z)**2)
    path_length = deltas.sum()
    euclidean = np.sqrt((x[-1] - x[0])**2 + (z[-1] - z[0])**2)
    if path_length == 0:
        return np.nan
    return euclidean / path_length

# Smoothing function
def smooth_trajectory(trj, window_length=240, polyorder=2):
    if len(trj) < window_length:
        return trj[['GameObjectPosX', 'GameObjectPosZ']].values
    x = trj['GameObjectPosX'].values
    z = trj['GameObjectPosZ'].values
    x_smooth = savgol_filter(x, window_length, polyorder)
    z_smooth = savgol_filter(z, window_length, polyorder)
    return np.column_stack((x_smooth, z_smooth))

# Get 20 random trial IDs
unique_ids = df_joined['UniqueTrialID'].dropna().unique()
sample_ids = random.sample(list(unique_ids), 20)

# Plot
plt.figure(figsize=(16, 12))
for i, trial_id in enumerate(sample_ids):
    trj = df_joined[df_joined['UniqueTrialID'] == trial_id]
    smoothed = smooth_trajectory(trj, window_length=51, polyorder=2)

    x_smooth, z_smooth = smoothed[:, 0], smoothed[:, 1]
    eff = path_efficiency(x_smooth, z_smooth)

    plt.subplot(4, 5, i + 1)
    plt.plot(trj['GameObjectPosX'], trj['GameObjectPosZ'], alpha=0.5, lw=1.2, label='Raw')
    plt.plot(x_smooth, z_smooth, lw=2, label='Smoothed')
    plt.scatter(trj['GameObjectPosX'].iloc[0], trj['GameObjectPosZ'].iloc[0], c='green', s=10)
    plt.scatter(trj['GameObjectPosX'].iloc[-1], trj['GameObjectPosZ'].iloc[-1], c='red', s=10)
    plt.axis('equal')
    plt.title(f'ID {trial_id}\nDirectedness: {eff:.2f}', fontsize=8)
    plt.xticks([])
    plt.yticks([])

plt.suptitle('Raw vs Smoothed Trajectories with Directedness', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()


In [ ]:
def path_efficiency(x, z):
    if len(x) < 2:
        return np.nan

    # Total path length
    deltas = np.sqrt(np.diff(x)**2 + np.diff(z)**2)
    path_length = deltas.sum()

    # Euclidean distance from start to end
    euclidean = np.sqrt((x[-1] - x[0])**2 + (z[-1] - z[0])**2)

    if path_length == 0:
        return np.nan

    return euclidean / path_length  # Between 0 (worst) and 1 (best)


In [ ]:
eff_results = []

window_length = 51  # Choose your current smoothing value

for trial_id in sample_ids:
    trj = df_joined[df_joined['UniqueTrialID'] == trial_id]
    
    x_raw = trj['GameObjectPosX'].values
    z_raw = trj['GameObjectPosZ'].values
    eff_raw = path_efficiency(x_raw, z_raw)

    smoothed = smooth_trajectory(trj, window_length=window_length, polyorder=2)
    x_smooth, z_smooth = smoothed[:, 0], smoothed[:, 1]
    eff_smooth = path_efficiency(x_smooth, z_smooth)

    eff_results.append({
        'UniqueTrialID': trial_id,
        'eff_raw': eff_raw,
        'eff_smooth': eff_smooth,
        'delta': eff_smooth - eff_raw
    })

import pandas as pd
eff_df = pd.DataFrame(eff_results)


In [ ]:
print(eff_df.describe())


In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(eff_df['eff_raw'], eff_df['eff_smooth'])
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
plt.xlabel('Efficiency (Raw)')
plt.ylabel('Efficiency (Smoothed)')
plt.title(f'Path Efficiency Before vs After Smoothing\n(window_length = {window_length})')
plt.tight_layout()
plt.show()


In [ ]:
window_length = 51
polyorder = 2

all_efficiency = []

for trial_id in df_joined['UniqueTrialID'].dropna().unique():
    trj = df_joined[df_joined['UniqueTrialID'] == trial_id]
    
    if len(trj) < window_length:
        continue  # skip short trajectories

    x_raw = trj['GameObjectPosX'].values
    z_raw = trj['GameObjectPosZ'].values
    eff_raw = path_efficiency(x_raw, z_raw)

    smoothed = smooth_trajectory(trj, window_length=window_length, polyorder=polyorder)
    x_smooth, z_smooth = smoothed[:, 0], smoothed[:, 1]
    eff_smooth = path_efficiency(x_smooth, z_smooth)

    all_efficiency.append({
        'UniqueTrialID': trial_id,
        'FlyID': trj['FlyID'].values[0],
        'Angle': trj['Angle'].values[0],
        'eff_raw': eff_raw,
        'eff_smooth': eff_smooth,
        'delta': eff_smooth - eff_raw,
        'n_points': len(trj)
    })

eff_df_all = pd.DataFrame(all_efficiency)


In [ ]:
# Group by FlyID
eff_by_ant = (
    eff_df_all.groupby('FlyID')[['eff_raw', 'eff_smooth', 'delta']]
              .mean()
              .sort_values('eff_smooth', ascending=False)
              .reset_index()
)

# Plot smoothed efficiency
plt.figure(figsize=(10, 4))
sns.barplot(data=eff_by_ant, x='FlyID', y='eff_smooth')
plt.xticks(rotation=45)
plt.ylabel('Mean Smoothed Efficiency')
plt.title('Path Efficiency per Ant (after smoothing)')
plt.ylim(0, 1)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(eff_df_all['eff_smooth'].dropna(), bins=30, range=(0, 1), edgecolor='black')
plt.xlabel('Path Efficiency (Smoothed)')
plt.ylabel('Number of Trials')
plt.title('Distribution of Path Efficiency Across All Trials')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(eff_df_all['eff_raw'].dropna(), bins=30, range=(0, 1), edgecolor='black')
plt.title('Raw Directedness')
plt.xlabel('Directedness')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
plt.hist(eff_df_all['eff_smooth'].dropna(), bins=30, range=(0, 1), edgecolor='black')
plt.title('Smoothed Directedness')
plt.xlabel('Directedness')

plt.suptitle('Efficiency Distribution: Raw vs Smoothed')
plt.tight_layout()
plt.show()


In [ ]:
eff_by_angle = (
    eff_df_all.groupby('Angle')['eff_smooth']
              .agg(['mean', 'std', 'count'])
              .reset_index()
              .rename(columns={'mean': 'mean_efficiency', 'std': 'std_efficiency'})
)


In [ ]:
plt.figure(figsize=(8, 5))
plt.errorbar(
    eff_by_angle['Angle'],
    eff_by_angle['mean_efficiency'],
    yerr=eff_by_angle['std_efficiency'],
    fmt='o-', capsize=4
)
plt.ylim(0, 1)
plt.xlabel('Platform Angle (deg)')
plt.ylabel('Mean Path Efficiency (Smoothed)')
plt.title('Path Efficiency vs Platform Angle')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Get relevant fields from df_joined (only one row per trial)
trial_meta = df_joined.drop_duplicates('UniqueTrialID')[[
    'UniqueTrialID', 'GoalReachedTime', 'is_correct', 'FlyID',
]]

df_combined = []
# Merge into efficiency data
df_combined = pd.merge(eff_df_all, trial_meta, on='UniqueTrialID', how='left')

# Clean types
df_combined['GoalReachedTime'] = pd.to_numeric(df_combined['GoalReachedTime'], errors='coerce')


In [ ]:
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df_combined, x='GoalReachedTime', y='eff_smooth', alpha=0.5)
sns.regplot(data=df_combined, x='GoalReachedTime', y='eff_smooth', scatter=False, color='red')
plt.xlabel('Decision Time (s)')
plt.ylabel('Directedness (Path Efficiency)')
plt.title('Directedness vs Decision Time')
plt.tight_layout()
plt.show()


In [ ]:
# Create 5 quantile-based bins (you can adjust to 4, 6, etc.)
df_combined['decision_time_bin'] = pd.qcut(df_combined['GoalReachedTime'], q=10, duplicates='drop')


In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df_combined, x='decision_time_bin', y='eff_smooth')
plt.xlabel('Decision Time (Quantile Bins)')
plt.ylabel('Directedness (Path Efficiency)')
plt.title('Directedness by Decision Time (Equal Count Bins)')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()


In [ ]:
# Group by decision time bin and compute mean accuracy
acc_by_bin = (
    df_combined.groupby('decision_time_bin')['is_correct']
               .mean()
               .reset_index(name='accuracy')
)



In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(data=acc_by_bin, x='decision_time_bin', y='accuracy')
plt.ylim(0, 1)
plt.xlabel('Decision Time (Quantile Bins)')
plt.ylabel('Proportion Correct')
plt.title('Accuracy by Decision Time (Equal Count Bins)')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(data=df_combined, x='is_correct', y='GoalReachedTime')
plt.xlabel('Correct Decision')
plt.ylabel('Decision Time (s)')
plt.title('Decision Time vs Accuracy')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(data=df_combined, x='is_correct', y='eff_smooth')
plt.xlabel('Correct Decision')
plt.ylabel('Directedness')
plt.title('Directedness vs Accuracy')
plt.tight_layout()
plt.show()


In [ ]:
df_combined


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Clean and convert before plotting
df_plot = df_combined.dropna(subset=['eff_smooth', 'is_correct', 'Angle']).copy()

# Convert to bool safely
df_plot['is_correct'] = df_plot['is_correct'].astype(bool)

# Label the correctness
df_plot['Correct'] = df_plot['is_correct'].map({True: 'Correct', False: 'Incorrect'})

# Sort angles for consistent layout
angles = sorted(df_plot['Angle'].dropna().unique())
n_cols = 3
n_rows = int(np.ceil(len(angles) / n_cols))

# Create plot
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 4), squeeze=False)

for i, angle in enumerate(angles):
    df_angle = df_plot[df_plot['Angle'] == angle]
    unique_labels = df_angle['Correct'].dropna().unique()

    ax = axes[i // n_cols, i % n_cols]
    
    if len(unique_labels) < 2:
        ax.text(0.5, 0.5, f"Not enough data\nfor angle {int(angle)}°",
                ha='center', va='center', fontsize=10)
        ax.axis('off')
    else:
        sns.boxplot(data=df_angle, x='Correct', y='eff_smooth',
                    order=['Incorrect', 'Correct'], ax=ax)
        ax.set_title(f'Angle: {int(angle)}°')
        ax.set_xlabel('')
        ax.set_ylabel('Directedness')
        ax.set_ylim(0.5, 1)

plt.suptitle('Directedness vs Accuracy per Angle', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()


In [ ]:
# Create quantile-based bins for directedness
df_combined['directedness_bin'] = pd.qcut(df_combined['eff_smooth'], q=15, duplicates='drop')
# Extract bin intervals and compute midpoints
df_combined['directedness_bin_mid'] = df_combined['directedness_bin'].apply(lambda x: x.mid)


In [ ]:
# Group by bin midpoints
bin_stats = (
    df_combined.groupby('directedness_bin_mid')['GoalReachedTime']
               .agg(['mean', 'std', 'count'])
               .reset_index()
               .rename(columns={'mean': 'mean_decision_time', 'std': 'std_decision_time'})
)

plt.figure(figsize=(8, 5))
plt.errorbar(
    bin_stats['directedness_bin_mid'],
    bin_stats['mean_decision_time'],
    yerr=bin_stats['std_decision_time'],
    fmt='o-', capsize=4
)
plt.xlabel('Directedness (Bin Midpoint)')
plt.ylabel('Decision Time (s)')
plt.title('Decision Time vs Directedness')
plt.tight_layout()
plt.show()


In [ ]:
acc_by_dir = (
    df_combined.groupby('directedness_bin_mid')['is_correct']
               .mean()
               .reset_index(name='accuracy')
)

plt.figure(figsize=(8, 5))
sns.lineplot(data=acc_by_dir, x='directedness_bin_mid', y='accuracy', marker='o')
plt.ylim(0, 1)
plt.xlabel('Directedness (Bin Midpoint)')
plt.ylabel('Accuracy (Proportion Correct)')
plt.title('Accuracy vs Directedness')
plt.tight_layout()
plt.show()


In [ ]:
# First, drop NaNs to avoid issues
df_angle_bins = df_combined.dropna(subset=['eff_raw', 'GoalReachedTime', 'is_correct', 'Angle']).copy()


# Create empty list to store per-angle binned data
binned_rows = []

# Apply qcut per angle
for angle, df_angle in df_angle_bins.groupby('Angle'):
    try:
        # Bin directedness into 5 equal-count bins
        bins = pd.qcut(df_angle['eff_raw'], q=9, duplicates='drop')
        midpoints = bins.apply(lambda x: x.mid)

        df_angle = df_angle.copy()
        df_angle['directedness_bin'] = bins
        df_angle['directedness_bin_mid'] = midpoints
        binned_rows.append(df_angle)
    except ValueError:
        # Skip angle if not enough data for binning
        continue

# Combine all binned data
df_binned = pd.concat(binned_rows)


In [ ]:
plt.figure(figsize=(10, 6))

for angle, df in df_binned.groupby('Angle'):
    grouped = df.groupby('directedness_bin_mid')['GoalReachedTime'].mean().reset_index()
    plt.plot(grouped['directedness_bin_mid'], grouped['GoalReachedTime'], marker='o', label=f'Angle {angle:.0f}°')

plt.xlabel('Directedness (Bin Midpoint)')
plt.ylabel('Decision Time (s)')
plt.title('Decision Time vs Directedness (per Angle)')
plt.legend(title='Platform Angle', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
df_combined

In [ ]:
plt.figure(figsize=(10, 6))

for angle, df in df_binned.groupby('Angle'):
    grouped = df.groupby('directedness_bin_mid')['is_correct'].mean().reset_index()
    plt.plot(grouped['directedness_bin_mid'], grouped['is_correct'], marker='o', label=f'Angle {angle:.0f}°')

plt.xlabel('Directedness (Bin Midpoint)')
plt.ylabel('Accuracy (Proportion Correct)')
plt.title('Accuracy vs Directedness (per Angle)')
plt.ylim(0, 1)
plt.legend(title='Platform Angle', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# Drop NaNs to avoid errors
df_dec_bins = df_combined.dropna(subset=['eff_smooth', 'GoalReachedTime', 'is_correct', 'Angle']).copy()

# Prepare storage
binned_decision_rows = []

# Apply qcut per angle
for angle, df_angle in df_dec_bins.groupby('Angle'):
    try:
        bins = pd.qcut(df_angle['GoalReachedTime'], q=5, duplicates='drop')
        midpoints = bins.apply(lambda x: x.mid)

        df_angle = df_angle.copy()
        df_angle['decision_time_bin'] = bins
        df_angle['decision_time_bin_mid'] = midpoints
        binned_decision_rows.append(df_angle)
    except ValueError:
        # Not enough data for binning → skip
        continue

# Combine
df_dec_binned = pd.concat(binned_decision_rows)


In [ ]:
plt.figure(figsize=(10, 6))

for angle, df in df_dec_binned.groupby('Angle'):
    grouped = df.groupby('decision_time_bin_mid')['eff_smooth'].mean().reset_index()
    plt.plot(grouped['decision_time_bin_mid'], grouped['eff_smooth'], marker='o', label=f'Angle {angle:.0f}°')

plt.xlabel('Decision Time (Bin Midpoint)')
plt.ylabel('Directedness (Path Efficiency)')
plt.title('Directedness vs Decision Time (per Angle)')
plt.legend(title='Platform Angle', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))

for angle, df in df_dec_binned.groupby('Angle'):
    grouped = df.groupby('decision_time_bin_mid')['is_correct'].mean().reset_index()
    plt.plot(grouped['decision_time_bin_mid'], grouped['is_correct'], marker='o', label=f'Angle {angle:.0f}°')

plt.xlabel('Decision Time (Bin Midpoint)')
plt.ylabel('Accuracy (Proportion Correct)')
plt.title('Accuracy vs Decision Time (per Angle)')
plt.ylim(0, 1)
plt.legend(title='Platform Angle', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
df_joined['ConfigFile'].unique()

In [ ]:
df_joined.columns

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
from scipy.ndimage import gaussian_filter
import math

# Binning parameters
n_xbins, n_zbins = 64, 64
x_range = (-60, 60)
z_range = (0, 60)
xedges = np.linspace(*x_range, n_xbins + 1)
zedges = np.linspace(*z_range, n_zbins + 1)
x_centers = 0.5 * (xedges[:-1] + xedges[1:])
z_centers = 0.5 * (zedges[:-1] + zedges[1:])

def get_right_goal(angle_deg, radius=60.0):
    half_angle_rad = np.radians(angle_deg / 2.0)
    return radius * np.sin(half_angle_rad), radius * np.cos(half_angle_rad)

# Filter configs
excluded_config = "BinaryChoice_constantSize_30deg_BlackCylinder_BlackCylinder.json"
df_filt = df_joined[df_joined["ConfigFile"] != excluded_config]
df_filt = df_filt.dropna(subset=["is_correct", "Angle", "GameObjectPosX", "GameObjectPosZ"])

# Unique angles
angles = sorted(df_filt["Angle"].dropna().unique())
nrows = len(angles)
ncols = 2  # correct / incorrect

fig, axes = plt.subplots(nrows, ncols, figsize=(12, nrows * 3.5), sharex=True, sharey=True)

for row_idx, angle in enumerate(angles):
    angle_df = df_filt[df_filt["Angle"] == angle]
    right_x, right_z = get_right_goal(angle)

    for col_idx, is_correct in enumerate([True, False]):
        ax = axes[row_idx, col_idx] if nrows > 1 else axes[col_idx]
        sub_df = angle_df[angle_df["is_correct"] == is_correct]

        # Mirror data
        x = sub_df["GameObjectPosX"].values
        z = sub_df["GameObjectPosZ"].values
        x_all = np.concatenate([x, -x])
        z_all = np.concatenate([z, z])

        valid_mask = (z_all >= 0) & (z_all <= right_z)
        x_all = x_all[valid_mask]
        z_all = z_all[valid_mask]

        # 2D histogram
        H, _, _ = np.histogram2d(x_all, z_all, bins=(n_xbins, n_zbins), range=[x_range, z_range])
        H_plot = H.T
        for z_idx in range(H_plot.shape[0]):
            row = H_plot[z_idx]
            if row.max() > row.min():
                H_plot[z_idx] = (row - row.min()) / (row.max() - row.min())

        H_blurred = gaussian_filter(H_plot, sigma=1.0, mode="nearest")

        # Get peak X per Z
        x_peakline = []
        for z_idx in range(H_blurred.shape[0]):
            row = H_blurred[z_idx]
            pos_mask = x_centers >= 0
            row_pos = row[pos_mask]
            x_pos = x_centers[pos_mask]
            if row_pos.sum() == 0:
                x_peakline.append(np.nan)
            else:
                x_peakline.append(x_pos[np.argmax(row_pos)])
        x_peakline = np.array(x_peakline)

        model_right = model_left = None
        skip_fit = np.isclose(angle, 180)

        if not skip_fit:
            # Fit bifurcation
            z_mask = z_centers <= 57
            z_fit = z_centers[z_mask]
            x_fit = x_peakline[z_mask]
            z_candidates = np.linspace(0, 55, 200)

            best_zc, best_error = None, np.inf
            for z_c in z_candidates:
                slope = right_x / (right_z - z_c)
                model = np.array([slope * (z - z_c) if z >= z_c else 0 for z in z_fit])
                valid = ~np.isnan(x_fit)
                error = np.mean((x_fit[valid] - model[valid]) ** 2)
                if error < best_error:
                    best_error = error
                    best_zc = z_c

            slope = right_x / (right_z - best_zc)
            model_right = np.array([slope * (z - best_zc) if z >= best_zc else 0 for z in z_centers])
            model_left = -model_right

        ax.imshow(H_blurred, origin="lower", aspect="auto", cmap="viridis",
                  extent=[xedges[0], xedges[-1], zedges[0], zedges[-1]])

        if not skip_fit:
            ax.plot(model_right, z_centers, color="red", lw=2)
            ax.plot(model_left, z_centers, color="red", lw=2)

            # Arc
            angle_rad = np.arctan(slope)
            angle_deg_fit = np.degrees(angle_rad)
            arc_angle = 2 * angle_deg_fit
            arc = Arc((0, best_zc), 10, 10, theta1=90 - angle_deg_fit, theta2=90 + angle_deg_fit,
                      color="white", lw=2)
            ax.add_patch(arc)
            ax.text(0, best_zc + 6, f"{arc_angle:.1f}°", ha="center", va="bottom",
                    color="white", fontsize=9)


        # Labels
        label = "Correct" if is_correct else "Incorrect"
        ax.set_title(f"{label} – {int(angle)}°")
        if col_idx == 0:
            ax.set_ylabel("Z Position (cm)")
        if row_idx == nrows - 1:
            ax.set_xlabel("X Position (cm)")
        ax.set_xlim(-60, 60)
        ax.set_ylim(0, 60)
        ax.set_aspect("equal")

plt.suptitle("Occupancy Maps by Angle – Correct vs Incorrect", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.95)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from scipy.ndimage import gaussian_filter1d

# --- Smoothing Function ---
def smooth_trajectory(trj, window_length=51, polyorder=2):
    if len(trj) < window_length:
        return trj[['GameObjectPosX', 'GameObjectPosZ']].values
    x = trj['GameObjectPosX'].values
    z = trj['GameObjectPosZ'].values
    x_smooth = savgol_filter(x, window_length, polyorder)
    z_smooth = savgol_filter(z, window_length, polyorder)
    return np.column_stack((x_smooth, z_smooth))

# --- Metric Calculations ---
def compute_metrics_for_trial(trj_df):
    smoothed = smooth_trajectory(trj_df)
    if len(smoothed) < 2:
        return None

    x, z = smoothed[:, 0], smoothed[:, 1]
    diffs = np.diff(smoothed, axis=0)
    step_lengths = np.linalg.norm(diffs, axis=1)
    path_length = np.sum(step_lengths)
    
    start = smoothed[0]
    end = smoothed[-1]
    straight_distance = euclidean(start, end)
    
    # Path efficiency
    eff = straight_distance / path_length if path_length > 0 else np.nan
    tortuosity = path_length / straight_distance if straight_distance > 0 else np.nan

    # Heading angles (in radians)
    headings = np.arctan2(diffs[:, 1], diffs[:, 0])
    d_heading = np.diff(headings)
    d_heading = (d_heading + np.pi) % (2 * np.pi) - np.pi  # wrap to [-π, π]
    
    # Wiggliness: sum of absolute heading changes
    wiggliness = np.nansum(np.abs(d_heading))

    # Turns: sign changes in d_heading
    sign_changes = np.sum(np.diff(np.sign(d_heading)) != 0)

    # Ideal heading vector
    ideal_vec = end - start
    ideal_angle = np.arctan2(ideal_vec[1], ideal_vec[0])
    angular_deviation = np.mean(np.abs((headings - ideal_angle + np.pi) % (2 * np.pi) - np.pi))

    # Lateral deviation
    if np.linalg.norm(ideal_vec) > 0:
        unit_vec = ideal_vec / np.linalg.norm(ideal_vec)
        lateral_offsets = []
        for point in smoothed:
            proj_length = np.dot(point - start, unit_vec)
            proj_point = start + proj_length * unit_vec
            deviation = np.linalg.norm(point - proj_point)
            lateral_offsets.append(deviation)
        lateral_deviation = np.mean(lateral_offsets)
    else:
        lateral_deviation = np.nan

    return {
        'UniqueTrialID': trj_df['UniqueTrialID'].iloc[0],
        'is_correct': trj_df['is_correct'].iloc[0],
        'FlyID': trj_df['FlyID'].iloc[0],
        'Angle': trj_df['Angle'].iloc[0],
        'efficiency': eff,
        'tortuosity': tortuosity,
        'wiggliness': wiggliness,
        'lateral_deviation': lateral_deviation,
        'num_turns': sign_changes,
        'angular_deviation': angular_deviation
    }

# --- Apply to All Trials ---
df_valid = df_joined.dropna(subset=['GameObjectPosX', 'GameObjectPosZ', 'UniqueTrialID', 'is_correct'])
metrics = []

for trial_id, trj_df in df_valid.groupby('UniqueTrialID'):
    result = compute_metrics_for_trial(trj_df)
    if result:
        metrics.append(result)

df_metrics = pd.DataFrame(metrics)


In [ ]:
df_metrics['is_correct'].unique()


In [ ]:
df_metrics['is_correct'].apply(lambda x: 'Correct' if x == 1.0 else 'Incorrect')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Map correctness to readable labels
df_metrics['Correct'] = df_metrics['is_correct'].apply(lambda x: 'Correct' if x == 1.0 else 'Incorrect')

# Metrics to include
metrics_to_plot = [
    'efficiency',           # Path efficiency
    'tortuosity',           # Total distance / straight-line distance
    'wiggliness',           # Mean change in heading
    'lateral_deviation',    # Average lateral deviation from ideal path
    'num_turns',            # Number of sharp heading reversals
    'angular_deviation'     # Total angular deviation
]

# Create subplots
n = len(metrics_to_plot)
fig, axes = plt.subplots(n, 1, figsize=(8, 3.5 * n), sharex=False)

# Plot boxplot + stripplot for each metric
for i, metric in enumerate(metrics_to_plot):
    ax = axes[i]
    sns.boxplot(data=df_metrics, x='Correct', y=metric, ax=ax, palette='Set2')
    sns.stripplot(data=df_metrics, x='Correct', y=metric, ax=ax, color='black', size=2, alpha=0.4, jitter=True)
    ax.set_title(metric.replace('_', ' ').title(), fontsize=12)
    ax.set_xlabel('')
    ax.set_ylabel('')

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Define metrics to plot
metrics = [
    'efficiency',           # Path efficiency
    'tortuosity',           # Total distance / straight-line distance
    'wiggliness',           # Mean change in heading
    'lateral_deviation',    # Average lateral deviation from ideal path
    'num_turns',            # Number of sharp heading reversals
    'angular_deviation'     # Total angular deviation
]

n_rows = 2
n_cols = 3
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 8), sharex=False)

for i, metric in enumerate(metrics):
    ax = axes[i // n_cols, i % n_cols]

    # Clip y-values to 1st–99th percentile for visual clarity
    lower, upper = np.percentile(df_metrics[metric], [1, 99])
    clipped_df = df_metrics[(df_metrics[metric] >= lower) & (df_metrics[metric] <= upper)]

    sns.boxplot(data=clipped_df, x='Correct', y=metric, ax=ax, palette='Set2')
    ax.set_title(metric.replace('_', ' ').title())
    ax.set_ylabel('')
    ax.set_xlabel('')

plt.suptitle("Trajectory Metrics by Correct vs Incorrect Decisions", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()


In [ ]:
def detect_significant_turns(rot_smoothed, threshold_rad=np.pi/4):
    """
    Detect turns when sign changes AND accumulated turn since last change exceeds threshold.
    """
    turn_indices = []
    last_sign = None
    accumulated_turn = 0
    last_idx = 0

    for i in range(1, len(rot_smoothed)):
        delta = rot_smoothed[i] - rot_smoothed[i - 1]
        delta = (delta + np.pi) % (2 * np.pi) - np.pi  # wrap to [-π, π]
        current_sign = np.sign(delta)
        
        if last_sign is None:
            last_sign = current_sign
            continue

        accumulated_turn += delta

        # If sign changes and we've turned enough → it's a turn
        if current_sign != 0 and current_sign != last_sign and abs(accumulated_turn) >= threshold_rad:
            turn_indices.append(i)
            accumulated_turn = 0
            last_sign = current_sign
            last_idx = i
        else:
            last_sign = current_sign  # keep tracking

    return np.array(turn_indices)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

turn_change_indices = detect_significant_turns(rot_smoothed, threshold_rad=np.pi/4)


# --- Plotting ---
fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# 1. Heading angles
ax[0].plot(rot_unwrapped, label="Unwrapped Raw Heading", alpha=0.5)
ax[0].plot(rot_smoothed, label="Smoothed Heading", linewidth=2)
ax[0].scatter(turn_change_indices, rot_smoothed[turn_change_indices], color='red', label="Turns", zorder=5)
ax[0].legend()
ax[0].set_ylabel("Heading (rad)")
ax[0].set_title(f"Heading vs. Turns – Trial {trial_id}")

# 2. Derivative of heading (turn rate)
ax[1].plot(d_rot, label="d(Heading)/dt", color='gray')
ax[1].hlines(0, 0, len(d_rot), linestyle='--', color='black')
ax[1].scatter(turn_change_indices[:-1], d_rot[turn_change_indices[:-1]], color='red', label="Turn Points")
ax[1].set_ylabel("Turn Rate")
ax[1].set_xlabel("Frame")
ax[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
turn_change_indices = detect_significant_turns(rot_smoothed, threshold_rad=np.pi/4)

# Smooth trajectory
from scipy.signal import savgol_filter
def smooth_trajectory(trj, window_length=51, polyorder=2):
    if len(trj) < window_length:
        return trj[['GameObjectPosX', 'GameObjectPosZ']].values
    x = trj['GameObjectPosX'].values
    z = trj['GameObjectPosZ'].values
    x_smooth = savgol_filter(x, window_length, polyorder)
    z_smooth = savgol_filter(z, window_length, polyorder)
    return np.column_stack((x_smooth, z_smooth))

smoothed_traj = smooth_trajectory(trj_df)

# Plot
plt.figure(figsize=(6, 6))
plt.plot(smoothed_traj[:, 0], smoothed_traj[:, 1], label="Smoothed Trajectory", lw=2)
plt.scatter(smoothed_traj[0, 0], smoothed_traj[0, 1], c='green', label="Start")
plt.scatter(smoothed_traj[-1, 0], smoothed_traj[-1, 1], c='red', label="End")
plt.scatter(smoothed_traj[turn_change_indices, 0], smoothed_traj[turn_change_indices, 1],
            c='orange', s=40, label="Turn Points", zorder=5)

plt.legend()
plt.title(f"Trajectory with Detected Turns – Trial {trial_id}")
plt.xlabel("X Position")
plt.ylabel("Z Position")
plt.axis('equal')
plt.tight_layout()
plt.show()
